In [1]:
import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import matplotlib
matplotlib.use('SVG')

import utils.callbacks
import utils.model_builder
import utils.visualize
import utils.data_provider
import utils.metrics

import skimage.io
import sklearn.metrics

import scipy.stats
import pandas as pd

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import sys

import os
# Uncomment the following line if you don't have a GPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

Using TensorFlow backend.
/home/ubuntu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/ubuntu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
# constants
const_lr = 1e-4

tag = '01'

base_dir = "/home/ubuntu/data/"

# Output dirs
out_dir = base_dir + 'unet/experiments/' + tag + '/out'
tb_log_dir = base_dir + "unet/tensorboard/" + tag + "/"
chkpt_dir = base_dir + "unet/experiments/" + tag + "/checkpoints/"

os.makedirs(out_dir, exist_ok=True)
os.makedirs(tb_log_dir, exist_ok=True)
os.makedirs(chkpt_dir, exist_ok=True)

# Files
chkpt_file = chkpt_dir + "{epoch:04d}.hdf5"
csv_log_file = base_dir + "unet/experiments/" + tag + "/log.csv"

# Input dirs
train_dir_x = base_dir + 'unet/split/training/x/'
train_dir_y = base_dir + 'unet/split/training/y/'
val_dir = base_dir + "unet/split/validation/"

# Learning Settings
rescale_labels = False

epochs = 2

batch_size = 4
steps_per_epoch = 2 * 4 / batch_size

# make sure these number for to the validation set
val_batch_size = 10
val_steps = int(50 * 4 / val_batch_size)

# generator only params
dim1 = 256
dim2 = 256

bit_depth = 8

In [3]:
# build session running on GPU 1
configuration = tf.ConfigProto()
#configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    train_dir_x,
    train_dir_y,
    batch_size,
    bit_depth,
    dim1,
    dim2,
    rescale_labels
)

val_gen = utils.data_provider.single_data_from_images(
    val_dir + 'x/',
     val_dir + 'y/',
     val_batch_size,
     bit_depth,
     dim1,
     dim2,
     rescale_labels
)

Found 50 images belonging to 1 classes.
Found 50 images belonging to 1 classes.


In [4]:
# build model
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.summary()

loss = "categorical_crossentropy"
metrics = [keras.metrics.categorical_accuracy, utils.metrics.recall, utils.metrics.precision]

optimizer = keras.optimizers.RMSprop(lr = const_lr)

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# CALLBACKS
# save model after each epoch
callback_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpt_file,
    save_weights_only=True,
    save_best_only=False
)
callback_csv = keras.callbacks.CSVLogger(filename=csv_log_file)
# callback_splits_and_merges = utils.callbacks.SplitsAndMergesLoggerBoundary(
#     'images',
#     val_gen,
#     gen_calls = val_steps,
#     log_dir=tb_log_dir
# )

callbacks=[callback_model_checkpoint, callback_csv] #, callback_splits_and_merges]


/home/ubuntu/unet4nuclei/unet4nuclei/utils/model_builder.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/ubuntu/unet4nuclei/unet4nuclei/utils/model_builder.py:27: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/unet4nuclei/unet4nuclei/utils/model_builder.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/ubuntu/unet4nuclei/unet4nuclei/utils/model_builder.py:33: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/unet4nuclei/unet4nuclei/utils/model_bui

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

In [5]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose = 1
)

# visualize learning stats
#utils.visualize.visualize_learning_stats_boundary_hard(statistics, out_dir, metrics)

print('Done! :)')

Epoch 1/2
Found 100 images.
Found 100 annotations.
2/2 [==============================] - 56s 28s/step - loss: 265.1697 - categorical_accuracy: 0.5983 - recall: 0.9995 - precision: 1.9006 - val_loss: 271.8424 - val_categorical_accuracy: 0.7557 - val_recall: 1.0000 - val_precision: 6553600000000.0000
Epoch 2/2
2/2 [==============================] - 16s 8s/step - loss: 186.8770 - categorical_accuracy: 0.7389 - recall: 1.0000 - precision: 1.3629 - val_loss: 256.9592 - val_categorical_accuracy: 0.7556 - val_recall: 1.0000 - val_precision: 6553600000000.0000
Done! :)
